In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import operator

In [2]:
filePath = "data/ml-100k/u1.base"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
datapoints = [[int(i) for i in data.split("\t")] for data in raw_data]

# indexing on users/movies starts at 1, reset to index from 0, this will be important when we do testing
datapoints = np.array([[row[0], row[1], row[2], row[3]] for row in datapoints])
np.random.shuffle(datapoints)

# Used in precision eval... not rly needed given datapoints
user_items = defaultdict(list)
for row in datapoints:
    user_items[row[0]].append(row[1])

user_ids = set([datapoint[0] for datapoint in datapoints])
n_users = len(user_ids)
max_user = max(user_ids) + 1
print("n users:", n_users)
items_ids = set([datapoint[1] for datapoint in datapoints])
n_items = len(items_ids)
max_item = max(items_ids) + 1
print("n items:", n_items)

n users: 943
n items: 1650


In [3]:
import random

# sanity check vs datapoints
user_item_cnts = {}
for user, movies in user_items.items():
    user_item_cnts[user] = len(movies)

top_users = dict(sorted(user_item_cnts.items(), key=operator.itemgetter(1), reverse=True)[:227])
print("User_ids with most watched movies: ", list(top_users.keys())[0:20], ". . .")

cold_start_users = random.sample(list(top_users.keys()), k=200) # Select k users randomly
print("\nNum cold start users: ", len(cold_start_users))

drift_users = random.sample(list(top_users.keys()), k=200)
print("\nDrift user ids: ", drift_users)

User_ids with most watched movies:  [655, 405, 450, 537, 416, 846, 682, 13, 880, 429, 896, 592, 796, 758, 561, 435, 551, 804, 474, 889] . . .

Num cold start users:  200

Drift user ids:  [463, 763, 621, 878, 758, 806, 913, 934, 504, 843, 825, 276, 654, 207, 484, 665, 864, 130, 455, 896, 862, 334, 899, 407, 622, 445, 773, 474, 268, 537, 882, 715, 749, 795, 194, 496, 13, 521, 774, 541, 339, 892, 497, 782, 536, 545, 291, 601, 815, 514, 886, 927, 660, 354, 385, 712, 399, 456, 303, 398, 721, 807, 907, 664, 608, 943, 642, 707, 903, 495, 305, 567, 85, 450, 682, 593, 757, 894, 346, 645, 7, 548, 897, 327, 671, 524, 629, 201, 666, 454, 222, 378, 901, 279, 374, 59, 840, 870, 271, 488, 435, 648, 561, 627, 727, 234, 922, 62, 711, 624, 606, 18, 851, 846, 788, 848, 532, 592, 699, 639, 299, 655, 1, 447, 405, 551, 751, 109, 328, 151, 178, 693, 503, 416, 883, 379, 738, 487, 618, 308, 881, 790, 486, 342, 577, 661, 458, 468, 586, 766, 373, 663, 452, 311, 453, 716, 347, 90, 698, 363, 919, 542, 269, 709, 9

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


def movie_genres(movie_ids):
    file = "data/ml-100k/u.item" 
    movies = pd.read_csv(file, sep='|', header=None, encoding='UTF-8')
    drop_cols = [2, 3, 4]
    movies = movies.drop(drop_cols, axis=1)
    col_names = ["movie_id", "title", "Unk", "Action", "Adventure", "Animation",
                  "Childrens", "Comedy", "Crime", "Documentary", "Drama",
                  "Fantasy", "Film_Noir", "Horror", "Musical", "Mystery",
                  "Romance", "Sci_Fi", "Thriller", "War", "Western"]
    movies.columns=col_names
    movies.head()
    return movies[movies['movie_id'].isin(movie_ids)].drop('title', axis=1)

def order_movies(userId, movie_split=60):
    file = "data/ml-100k/u1.base"
    with open(filePath, "rt") as dataPath:
        raw_data = dataPath.read().splitlines()
    datapoints = [[int(i) for i in data.split("\t")] for data in raw_data]
    datapoints = np.array([[row[0], row[1], row[3]] for row in datapoints if row[0] == userId])
    sorted(datapoints, key = lambda x: int(x[2]))
    movie_set1, movie_set2 = [], []
    count = 0
    for row in datapoints:
        count += 1
        if count <= movie_split:
            movie_set1.append(row[1])
        elif movie_split < count <= movie_split*2:
            movie_set2.append(row[1])
        else:
            break
    if len(movie_set1) != len(movie_set2):
        print("Error: Movie set lengths differ!")
        return -1, -1
    return movie_set1, movie_set2

def genre_distribution(genre_matrix):
    col_sums = np.sum(genre_matrix.drop('movie_id', axis=1), axis=0)
    distribution = col_sums/sum(col_sums)
    return distribution

def getCosineSimilarity(userId, movie_split):
    # Get list of viewed movies ordered by timestamp
    movie_set1, movie_set2 = order_movies(userId, movie_split)
    # Get genre information for sets of movies
    genre_matrix1 = movie_genres(movie_set1)
    genre_matrix2 = movie_genres(movie_set2)
    # Get genre distribution vectors set1 vs set2
    dist1 = genre_distribution(genre_matrix1)
    dist2 = genre_distribution(genre_matrix2)
    # Calculate cosine similarity between vectors
    similarity = cosine_similarity(np.array(dist1).reshape(1,-1), np.array(dist2).reshape(1,-1))
    return similarity[0][0]
    

In [5]:
cosim_dict = {}
movie_split = 60
for user in drift_users:
    cosim_dict[user] = getCosineSimilarity(user, movie_split) # Slow since repeat file reads - fix if not lazy

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 3: invalid continuation byte

In [ ]:
lowest_cosim_ids = sorted(cosim_dict, key=cosim_dict.get)[0:5]
lowest_cosim_ids

In [ ]:
def softmax(x):
    # subtract max value to prevent overflow\n"
    return np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)))

# computes average and standard deviation MRR for samples
def mrr(R, U, V, samples):
    mrrs = []
    #print(R[samples, :][0])
    for s in samples:
        mrr = 0.0
        rating_count = np.sum(R[s, :] >= 3.0)
        if rating_count == 0:
            continue
        pred = np.matmul(U[s], V.T)
        predRank = np.argsort(np.argsort(-pred)) + 1
        rs, cs = R[s, :].nonzero()
        for j in cs:
            mrr += (1.0 / predRank[j])
        mrr /= rating_count
        mrrs.append(mrr)
    return np.mean(mrrs), np.std(mrrs)

def stratified_resample(weights):
    N = len(weights)
    # make N subdivisions, chose a random position within each one
    positions = (np.random.random(N) + range(N)) / N

    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes

In [ ]:
""" ONLY RUN IF TESTING ON DRIFT USERS ONLY """
filePath = "data/ml-100k/u1.base"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
datapoints = [[int(i) for i in data.split("\t")] for data in raw_data]

# indexing on users/movies starts at 1, reset to index from 0, this will be important when we do testing
drift_users = set(lowest_cosim_ids)
datapoints = np.array([[row[0], row[1], row[2], row[3]] for row in datapoints if row[0] in drift_users])
# timestamp sorted. So algo will go throw movies as if seeing user actions sequentially
datapoints = sorted(datapoints, key = lambda x: int(x[3]))


In [ ]:
n_particles = 2 
k = 5 
var = 0.5
particles = [(1 / n_particles, {"u": np.random.normal(size=(max_user, k)),
                                "v": np.random.normal(size=(max_item, k)),
                                "var_u": 1.0,
                                "var_i": 1.0}) for _ in range(n_particles)]

# get mean rating to make rating data centered at 0
#print(datapoints[:, 2])
#mean_rating = np.mean(datapoints[:, 2])
#data_store = {u_id: {row[1]: row[2] - mean_rating for row in datapoints if row[0] == u_id} for u_id in user_ids}


data_store = {u_id: {row[1]: row[2] for row in datapoints if row[0] == u_id} for u_id in drift_users}

user_history = {} # user_rating_history[user_id]["item_ids"], user_rating_history[user_id]["ratings"]
item_history = {} # item_rating_history[item_id]["user_ids"], item_rating_history[item_id]["ratings"]
ses = []
ctr = 0
ctr_hist = []
#U_trace_hist =  [] #[ np.linalg.norm(particles[0][1]['u'],'fro') ]
#V_trace_hist = [] # [ np.linalg.norm(particles[0][1]['v'],'fro') ]
# initialize drift test
drift1 = {}
drift2 = {}
rand1 = {}
rand2 = {}
user_cnt = {}
for u_id in data_store.keys():
    print(u_id)
    drift1[u_id] = []
    drift2[u_id] = []
    user_cnt[u_id] = 0
    rand1[u_id] = []
    rand2[u_id] = []

# what we're going to do here is assume that OUR system made the item recommendation and is observing the
# rating we have in the dataset ... it's sort of like we're starting at line 11
for _idx in range(len(datapoints)):

    # randomly get a user
    user_id = np.random.choice([i for i in data_store.keys()])
    user_items = [i for i in data_store[user_id].keys()]
    
    # highest rating this user has
    highest_rating = max(data_store[user_id].values())
    # get highest rated items
    highest_rated_items = [x for x in data_store[user_id].keys() if data_store[user_id][x] >= highest_rating]
    # get indices for items this user rated
    indices = np.array(user_items)
    
    # randomly select a particle
    random_particle = np.random.choice(range(n_particles))
    particle = particles[random_particle]
    
    # predict a rating only for the items rated by that user
    predicted_rating = np.dot(particle[1]["u"][user_id, :], particle[1]["v"][indices, :].T)

    # get the item id
    max_rating_ind = np.argmax(predicted_rating)
    item_id = [i for i in data_store[user_id].keys()][max_rating_ind]
    

    # add to ctr if possible
    if item_id in highest_rated_items:
        ctr += 1
    ctr_hist.append(ctr / (_idx + 1))
    
    # get the true rating
    rating = data_store[user_id][item_id]
    
    # For drift_users - track performance
    user_cnt[user_id] += 1
    if item_id in highest_rated_items and rating >= 3:
        if user_cnt[user_id] <= 60:
            drift1[user_id].append(1)
        elif 60 < user_cnt[user_id] <= 120:
            drift2[user_id].append(1)
    else:
        if user_cnt[user_id] <= 60:
            drift1[user_id].append(0)
        elif 60 < user_cnt[user_id] <= 120:
            drift2[user_id].append(0)
            
    # Random recommendation baseline
    rand_id = np.random.choice([i for i in data_store[user_id].keys()])
    if rand_id in highest_rated_items and rating > 3:
        if user_cnt[user_id] <= 60:
            rand1[user_id].append(1)
        elif 60 < user_cnt[user_id] <= 120:
            rand2[user_id].append(1)
    else:
        if user_cnt[user_id] <= 60:
            rand1[user_id].append(0)
        elif 60 < user_cnt[user_id] <= 120:
            rand2[user_id].append(0)
    
    # delete this item from this user
    del data_store[user_id][item_id]
    
    # delete the user from the data store if they have no reviews left
    if not data_store[user_id]:
        del data_store[user_id]
        
    error = predicted_rating[max_rating_ind] - rating
    se = error ** 2
    if _idx % 101 == 0:
        ses.append(se)
    if _idx % 1000== 0:
        print("squared error: {:.2f}".format(se))

    # line 17
    precision_u_i = []
    eta_u_i = []
    for particle in particles:
        if user_id not in user_history:
            precision_u_i.append(np.eye(k))
            eta_u_i.append(np.zeros(k))
        else:
            v_j = particle[1]["v"][user_history[user_id]["item_ids"], :]
            lambda_u_i = 1 / var * np.dot(v_j.T, v_j) + 1 / particle[1]["var_u"] * np.eye(k)

            precision_u_i.append(lambda_u_i)

            eta = np.sum(
                np.multiply(
                    v_j,
                    np.array(user_history[user_id]["ratings"]).reshape(-1, 1)
                ),
                axis=0
            )
            eta_u_i.append(eta.reshape(-1))

    # line 18
    weights = []
    mus = [1 / var * np.dot(np.linalg.inv(lambda_), eta) for lambda_, eta in zip(precision_u_i, eta_u_i)]
    for particle, mu, precision in zip(particles, mus, precision_u_i):
        v_j = particle[1]["v"][item_id, :]
        cov = 1 / var + np.dot(np.dot(v_j.T, precision), v_j)
        w = np.random.normal(
            np.dot(v_j.T, mu),
            cov
        )
        weights.append(w)
    normalized_weights = softmax(weights)
    
    # Generate trace plots of U, V
#     U_trace, V_trace = 0, 0
#     for particle, weight in zip(particles, normalized_weights):
#         U_trace += particle[1]["u"] * weight
#         V_trace += particle[1]["v"] * weight
#     U_trace_hist.append(np.linalg.norm(U_trace, 'fro'))
#     V_trace_hist.append(np.linalg.norm(V_trace, 'fro'))

    # line 19
    ds = [np.random.choice(range(n_particles), p=normalized_weights) for _ in range(n_particles)]
    p_prime = [{"u": np.copy(particles[d][1]["u"]),
                "v": np.copy(particles[d][1]["v"]),
                "var_u": particles[d][1]["var_u"],
                "var_i": particles[d][1]["var_i"]} for d in ds]
    for idx, (particle, precision, e) in enumerate(zip(p_prime, precision_u_i, eta_u_i)):

        # line 21
        v_j = particle["v"][item_id, :]
        add_to_precision = 1 / var * np.dot(v_j.reshape(-1, 1), v_j.reshape(1, -1))
        precision += add_to_precision

        add_to_eta = rating * v_j
        e += add_to_eta

        # line 22
        p_prime[idx]["u"][user_id, :] = np.random.multivariate_normal(
            1 / var * np.dot(np.linalg.inv(precision), e),
            np.linalg.inv(precision)
        )
        
        
        # line 24
        if item_id not in item_history:
            precision_v_i = np.eye(k)
            eta = np.zeros(k)
        else:
            u_i = particle["u"][item_history[item_id]["user_ids"], :]
            precision_v_i = 1 / var * \
                np.dot(u_i.T, u_i) + \
                1 / particle["var_i"] * np.eye(k)

            eta = np.sum(
                np.multiply(
                    u_i,
                    np.array(item_history[item_id]["ratings"]).reshape(-1, 1)
                ),
                axis=0
            )
        # line 25
        p_prime[idx]["v"][item_id, :] = np.random.multivariate_normal(
            1 / var * np.dot(np.linalg.inv(precision_v_i), eta),
            np.linalg.inv(precision_v_i)
        )

    # line 28
    particles = [(1 / n_particles, particle) for particle in p_prime]

    if user_id not in user_history:
        user_history[user_id] = {"item_ids": [], "ratings": []}
    if item_id not in item_history:
        item_history[item_id] = {"user_ids": [], "ratings": []}
    user_history[user_id]["item_ids"].append(item_id)
    user_history[user_id]["ratings"].append(rating)
    item_history[item_id]["user_ids"].append(user_id)
    item_history[item_id]["ratings"].append(rating)

np.save("results/train_Particles={}_k={}_var={}.npy".format(n_particles, k, var), particles)
np.save("results/train_MSE_hist_p={}_k={}_var={}.npy".format(n_particles, k, var), ses)   
    

In [ ]:
for user in drift1.keys():
    rand_sum = sum(rand1[user]) + sum(rand2[user])
    drift_sum = sum(drift1[user]) + sum(drift2[user])
    #print("random sum: {} | drift sum: {}".format(rand_sum, drift_sum))
    print("user: ", user, "  drift1:", sum(drift1[user]), "  rand1:", sum(rand1[user]))
    print("user: ", user, "  drift2:", sum(drift2[user]), "  rand2:", sum(rand2[user]))
    print()


In [ ]:
movies_rec = user_history[901]['item_ids']
movie_set1 = movies_rec[0:60]
movie_set2 = movies_rec[61:121]
print(len(movie_set1) == len(movie_set2))

genre_matrix1 = movie_genres(movie_set1)
genre_matrix2 = movie_genres(movie_set2)
dist1 = genre_distribution(genre_matrix1)
dist2 = genre_distribution(genre_matrix2)
similarity = cosine_similarity(np.array(dist1).reshape(1,-1), np.array(dist2).reshape(1,-1))[0][0]

print(dist1)
print()
print(dist2)
print()
print(dist1-dist2)
print(similarity)

In [ ]:
plt.plot(range(len(U_trace_hist)), U_trace_hist)
plt.title("$\|\|U\|\|_{Fro}^2$ at Each Sample")
plt.xlabel("Time Step")
plt.savefig("weird_U_norm_trace.png")
plt.show()


# plt.plot(range(len(V_trace_hist)), V_trace_hist)
# plt.title("$\|V\|_{Fro}^2$ at Each Sample")
# plt.xlabel("Time Step")
# plt.show()

In [ ]:
import random

def test(particles, data):
    
    user_ids = set([item[0] for item in data])
    items_ids = set([item[1] for item in data])
    data_store = {u_id: {row[1]: row[2] for row in data if row[0] == u_id} for u_id in user_ids}
    user_history = {} # user_rating_history[user_id]["item_ids"], user_rating_history[user_id]["ratings"]
    item_history = {} # item_rating_history[item_id]["user_ids"], item_rating_history[item_id]["ratings"]

    #Evaluation stuffs
    precision = 0
    ses = []
    precision_list = []
    
    for _idx in range(len(data)):
        
        # randomly get a user
        user_id = np.random.choice([i for i in data_store.keys()])
        user_items = [i for i in data_store[user_id].keys()]
        
        # get indices for items this user rated
        indices = np.array(user_items)
        
        # randomly select a particle
        random_particle = np.random.choice(range(len(particles)))
        particle = particles[random_particle]
        
        # predict a rating only for the items rated by that user
        predicted_rating = np.dot( particle[1]["u"][user_id, :], particle[1]["v"][indices, :].T)
        
        # get the item id
        max_rating_ind = np.argmax(predicted_rating)
        item_id = [i for i in data_store[user_id].keys()][max_rating_ind]
        
        # get the true rating
        rating = data_store[user_id][item_id]

        # delete this item from this user
        del data_store[user_id][item_id]

        # delete the user from the data store if they have no reviews left
        if not data_store[user_id]:
            del data_store[user_id]
          

        #pred = predicted_rating[max_rating_ind]
        pred = random.randint(1,5)

        if pred > 5:
            pred = 5
        elif pred < 1:
            pred = 1
    
        error = pred - rating
        se = error ** 2
        
        # Precision
        if pred >= 4 and rating >= 4:
            precision += 1
        if _idx % 10 == 0:
            ses.append(se)
        if _idx % 100 == 0:
            print("squared error: {:.2f}".format(se))
        if _idx in [10, 20, 40 ,60, 80, 120]:
                precision_list.append((_idx, precision))
        
        # line 17
        precision_u_i = []
        eta_u_i = []
        for particle in particles:
            if user_id not in user_history:
                precision_u_i.append(np.eye(k))
                eta_u_i.append(np.zeros(k))
            else:
                v_j = particle[1]["v"][user_history[user_id]["item_ids"], :]
                lambda_u_i = 1 / var * np.dot(v_j.T, v_j) + 1 / particle[1]["var_u"] * np.eye(k)

                precision_u_i.append(lambda_u_i)

                eta = np.sum(
                    np.multiply(
                        v_j,
                        np.array(user_history[user_id]["ratings"]).reshape(-1, 1)
                    ),
                    axis=0
                )
                eta_u_i.append(eta.reshape(-1))

        # line 18
        weights = []
        mus = [1 / var * np.dot(np.linalg.inv(lambda_), eta) for lambda_, eta in zip(precision_u_i, eta_u_i)]
        for particle, mu, precision in zip(particles, mus, precision_u_i):
            v_j = particle[1]["v"][item_id, :]
            cov = 1 / var + np.dot(np.dot(v_j.T, precision), v_j)
            w = np.random.normal(
                np.dot(v_j.T, mu),
                cov
            )
            weights.append(w)
        normalized_weights = softmax(weights)

        # line 19
        ds = [np.random.choice(range(n_particles), p=normalized_weights) for _ in range(n_particles)]
        p_prime = []
        for d in ds:
            p_prime.append((1 / n_particles, particles[d][1]))

        for particle, precision, e in zip(p_prime, precision_u_i, eta_u_i):

            # line 21
            v_j = particle[1]["v"][item_id, :]
            add_to_precision = 1 / var * np.dot(v_j.reshape(-1, 1), v_j.reshape(1, -1))
            precision += add_to_precision

            add_to_eta = rating * v_j
            e += add_to_eta

            # line 22
            particle[1]["u"][user_id, :] = np.random.multivariate_normal(
                1 / var * np.dot(np.linalg.inv(precision), e),
                np.linalg.inv(precision)
            )

            # line 24
            if item_id not in item_history:
                precision_v_i = np.eye(k)
                eta = np.zeros(k)
            else:
                u_i = particle[1]["u"][item_history[item_id]["user_ids"], :]
                precision_v_i = 1 / var * \
                    np.dot(u_i.T, u_i) + \
                    1 / particle[1]["var_i"] * np.eye(k)

                eta = np.sum(
                    np.multiply(
                        u_i,
                        np.array(item_history[item_id]["ratings"]).reshape(-1, 1)
                    ),
                    axis=0
                )
            # line 25
            particle[1]["v"][item_id, :] = np.random.multivariate_normal(
                1 / var * np.dot(np.linalg.inv(precision_v_i), eta),
                np.linalg.inv(precision_v_i)
            )

        # line 28
        particles = p_prime

        if user_id not in user_history:
            user_history[user_id] = {"item_ids": [], "ratings": []}
        if item_id not in item_history:
            item_history[item_id] = {"user_ids": [], "ratings": []}
        user_history[user_id]["item_ids"].append(item_id)
        user_history[user_id]["ratings"].append(rating)
        item_history[item_id]["user_ids"].append(user_id)
        item_history[item_id]["ratings"].append(rating)
    return ses, precision_list

In [ ]:
""" Cold Start Test """
filePath = "data/ml-100k/u1.base"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
    
data = [[int(i) for i in data.split("\t")] for data in raw_data]
data = np.array([[row[0], row[1], row[2], row[3]] for row in data if row[0] in cold_start_users])
#np.random.shuffle(data)

test_ses, ctr_list = test(particles, data)

#np.save("results/test_MSE_hist_p={}_k={}_var={}_run2.npy".format(n_particles, k, var), test_ses)   
# print()
for idx, ctr in ctr_list:
    print("idx: {}, ctr: {}".format(idx, ctr/len(cold_start_users)))


In [ ]:
print(ctr_list)

In [ ]:
filePath = "data/ml-100k/u1.test"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
    
data = [[int(i) for i in data.split("\t")] for data in raw_data]

# indexing on users/movies starts at 1, reset to index from 0, this will be important when we do testing
data = np.array([[row[0], row[1], row[2], row[3]] for row in data if row[0] in cold_start_users])
#data = np.array([[row[0], row[1], row[2], row[3]] for row in data])
#np.random.shuffle(data)

test_ses, ctr_list = test(particles, data)

#np.save("results/test_MSE_hist_p={}_k={}_var={}_run2.npy".format(n_particles, k, var), test_ses)   
print()
for idx, ctr in ctr_list:
    print("idx: {}, ctr: {}".format(idx, ctr))

In [ ]:
def moving_average(x):
    avgs = []
    for i, v in enumerate(x):
        avgs.append(np.sum(x[:i]) / i)
    return avgs

In [ ]:
mses = moving_average(test_ses)
print(mses[-1])

In [ ]:
m = np.array([2.44121342564, 2.40624590454, 2.29556823627, 2.30130174491, 2.19908182858])

avg = np.average(m)
std = np.std(m)
min_v = np.min(m)
max_v = np.max(m)
print("avg:", avg)
print("std:", std)
print("min:", min_v)
print("max:", max_v)

In [ ]:
mses = moving_average(ses)
plt.plot(range(len(mses)), mses)
plt.title("Train MSE: {:.2f} | Particles=2 K=5".format(mses[-1]))
plt.savefig("MSE_graph.png")
plt.show()

In [ ]:
# 1 user - get u vec 
# know how 
w = [.15, .25, .60 ]
l = [1, 2, 3]

x = np.random.random(l, w)

In [ ]:
plt.scatter(range(len(ctr_hist)), ctr_hist)
plt.title("cumulative take rate {:.2f}".format(ctr_hist[-1]))
plt.show()